### Reformat LPR behavioral data to have t0-t239
### While reformatting, divide data into 240 and 15 timepoints sets respectively

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, random, time
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/1_reformat/all_targets/LPR


In [3]:
# mac - phase I & II - LPR - before merging - chemical 53 only
complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/latest/before_merging/tall/df_per_chemical_53_tall.csv'

# mac - phase III - full
#complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/original/behavior/LPR/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23.csv'
# -> 240 unique variables and 215 unique chemical IDs

In [4]:
df_behav = pd.read_csv(complete_input_file_path, header = 0)
df_behav = df_behav.rename({'endpoint': 'timepoint'}, axis=1)
df_behav = df_behav.rename({'variable': 'timepoint'}, axis=1)

df_behav['chemical.id'] = df_behav['chemical.id'].astype(str)
# this recasting is needed for "df_select_1846 = df_select.loc[df_select['chemical.id'] == '1846',:]" later

display(df_behav.head())
display(df_behav.tail())

,chemical.id,bottle.id,conc,plate.id,well,timepoint,value
0,53,C96423,NaN,NaN,NaN,t0,NaN
1,53,C96423,0.0,9289.0,A06,t0,0.0
2,53,C96423,0.0,9289.0,A12,t0,0.0
3,53,C96423,0.0,9289.0,B06,t0,0.0
4,53,C96423,0.0,9289.0,B12,t0,0.0


,chemical.id,bottle.id,conc,plate.id,well,timepoint,value
46315,53,C96423,50.0,9290.0,F07,t239,0.0
46316,53,C96423,50.0,9290.0,G01,t239,26.4
46317,53,C96423,50.0,9290.0,G07,t239,4.4
46318,53,C96423,50.0,9290.0,H01,t239,0.0
46319,53,C96423,50.0,9290.0,H07,t239,17.6


In [5]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'timepoint', 'value']
df_select = df_behav.loc[:,columns_to_keep]

display(df_select.head())
display("number of unique chemical.id:" + str(len(np.unique(df_select['chemical.id']))))
display("number of unique timepoints:" + str(len(np.unique(df_select['timepoint']))))

,chemical.id,conc,plate.id,well,timepoint,value
0,53,NaN,NaN,NaN,t0,NaN
1,53,0.0,9289.0,A06,t0,0.0
2,53,0.0,9289.0,A12,t0,0.0
3,53,0.0,9289.0,B06,t0,0.0
4,53,0.0,9289.0,B12,t0,0.0


'number of unique chemical.id:1'

'number of unique timepoints:240'

In [6]:
nan = df_select[df_select['value'].isna()]
display(nan.head())

# [phase III] there is no nan in 'chemical.id', 'conc', 'plate.id', 'well', 'variable'

# don't drop na now for easier proceesing for now

,chemical.id,conc,plate.id,well,timepoint,value
0,53,NaN,NaN,NaN,t0,NaN
193,53,NaN,NaN,NaN,t1,NaN
386,53,NaN,NaN,NaN,t2,NaN
579,53,NaN,NaN,NaN,t3,NaN
772,53,NaN,NaN,NaN,t4,NaN


### Transpose time points 

In [7]:
# old using groupby
#'''
start_time = time.time()
       
def reformat(chemical_index, df_select, df_reformatted_240_timepoints, df_reformatted_15_timepoints):
    df_per_chemical = df_select.loc[df_select['chemical.id'] == chemical_index,:]
    #display (df_per_chemical.head())

    # Append chemical_plate_well as a unique identifier
    # takes long time (~1 min)
    df_per_chemical.insert(0, 'chemical_plate_well', df_per_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(df_per_chemical.chemical_plate_well):
        #print (str(cpw))
        per_cpw = df_per_chemical.loc[df_per_chemical.chemical_plate_well == cpw,:]
        per_cpw_grouped = per_cpw.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in per_cpw_grouped:
            concat_this = pd.DataFrame(
                    {
                    'chemical.id': np.unique(per_cpw['chemical.id']),
                    'plate.id': np.unique(per_cpw['plate.id']),
                    'well': np.unique(per_cpw['well']),
                    'chemical_plate_well': np.unique(per_cpw['chemical_plate_well']),
                    'conc': np.unique(per_cpw['conc'])
                    })
            
            timepoints_15 = False # init
            # rename timepoint columns if this is for 15 endpoints
            for time_point in np.arange(len(np.unique(group.timepoint))):
                if (len(np.unique(group.timepoint)) == 15):
                    timepoints_15 = True
                    time_point = time_point + 3
                timepoint = 't'+ str(time_point)
                concat_this = pd.concat([concat_this, pd.DataFrame({timepoint: per_cpw.value[per_cpw.timepoint == timepoint].values})],axis = 1)

            if (timepoints_15 == False):
                df_reformatted_240_timepoints = pd.concat([df_reformatted_240_timepoints, concat_this])
            else:
                df_reformatted_15_timepoints = pd.concat([df_reformatted_15_timepoints, concat_this])

    return df_reformatted_240_timepoints, df_reformatted_15_timepoints
########### end of def reformat(chemical_index, behav_select, df_reformatted):


df_reformatted_240_timepoints = pd.DataFrame()
df_reformatted_15_timepoints = pd.DataFrame()

full_devel = "full"
#full_devel = "devel"

chemical_id_from_here = np.unique(df_behav['chemical.id'])

if (full_devel == "devel"):
    randomly_chosen = random.sample(set(chemical_id_from_here), 2)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])

#chemical_id_from_here = ['1030', '1119']
chemical_id_from_here = ['53']
# 1119 chemical.id ->  15 timepoints
# 1030 chemical.id -> 240 timepoints

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

for chemical_index in chemical_id_from_here:
    print("\nchemical_index:" + str(chemical_index))

    df_reformatted_240_timepoints, df_reformatted_15_timepoints \
    = reformat(chemical_index, df_select, df_reformatted_240_timepoints, df_reformatted_15_timepoints)
    
    number_of_chemicals_processed += 1
    print_this = str(number_of_chemicals_processed) + " chemicals processed out of " + str(total_number_of_chemicals_to_processed)
    print(print_this)
    
    #display('number of unique chemical.id:', str(len(np.unique(df_reformatted['chemical.id']))))
    
    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Transposing time points is done. It took " + str(time_took)) 
# took 75 seconds in pnnl laptop for 1 chemical
# took 5~7 hrs in pnnl laptop for 196 chemicals
#'''


chemical_index:53
1 chemicals processed out of 1
Current Time = 19:12:15
Transposing time points is done. It took 80.0 seconds


In [8]:
display(df_reformatted_240_timepoints.head())
display(df_reformatted_15_timepoints.head())

,chemical.id,plate.id,well,chemical_plate_well,conc,t0,t1,t2,t3,t4,...,t230,t231,t232,t233,t234,t235,t236,t237,t238,t239
0,53,9289.0,A01,53_9289.0_A01,50.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,53,9289.0,A02,53_9289.0_A02,35.6,7.2,0.0,0.0,0.0,0.0,...,14.1,15.2,17.3,17.5,10.6,13.4,9.1,11.2,11.7,10.6
0,53,9289.0,A03,53_9289.0_A03,11.2,0.0,0.0,0.0,0.0,0.0,...,0.0,13.9,12.9,0.0,11.5,12.9,9.2,13.1,3.0,7.9
0,53,9289.0,A04,53_9289.0_A04,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,53,9289.0,A05,53_9289.0_A05,1.0,0.0,0.0,0.0,0.0,0.0,...,13.7,18.1,7.1,10.3,8.3,17.5,12.3,13.9,10.8,15.9


""


In [9]:
reformatted_data_filename = str(complete_input_file_path[:-4]) + "_wide_t0_t239_" + str(full_devel) + ".csv"
display ("reformatted_data_filename:", reformatted_data_filename)
df_reformatted_240_timepoints.to_csv(reformatted_data_filename, index=False)

reformatted_data_filename = str(complete_input_file_path[:-4]) + "_wide_t3_t17_" + str(full_devel) + ".csv"
display ("reformatted_data_filename:", reformatted_data_filename)
df_reformatted_15_timepoints.to_csv(reformatted_data_filename, index=False)

'reformatted_data_filename:'

'/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/latest/before_merging/tall/df_per_chemical_53_tall_wide_t0_t239_full.csv'

'reformatted_data_filename:'

'/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/latest/before_merging/tall/df_per_chemical_53_tall_wide_t3_t17_full.csv'